In [1]:
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    
import os
import re
import copy
import pandas as pd

In [2]:
# матрицу в вектор
def mat_to_vec(x):
    m = x.shape[0]*x.shape[1]
    tmp1 = np.zeros(m)

    c = 0
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            tmp1[c] = x[i,j]
            c +=1
    return tmp1


# создаем матрицу весов для одного изображения
def create_W(x):
    w = np.zeros([len(x),len(x)])
    for i in range(len(x)):
        for j in range(i,len(x)):
            if i == j:
                w[i,j] = 0 # обнуляем если номер строки и столбца совпадают, так получим обнуление по диагонали
            else:
                w[i,j] = x[i]*x[j]
                w[j,i] = w[i,j]
    return w

# Приводим изображение к виду состоящего из матрицы из единиц и минус единиц
def readImg2array(file,size, threshold = 145, invert = False): # threshold - порог, чтобы преобразовать все значения канала выше него в 1
    pilIN = Image.open(file).convert(mode="L")
    
    if invert == True:
        inverted_image = PIL.ImageOps.invert(pilIN)
        pilIN = inverted_image.resize(size)
    elif invert == False:
        pilIN = pilIN.resize(size)
    # pilIN.show()
    
    imgArray = np.asarray(pilIN,dtype=np.uint8)

    x = np.zeros(imgArray.shape,dtype=np.float)
    x[imgArray > threshold] = 1 # соответственно здесь и нужен этот порог, все что выше него становится 1
    x[x==0] = -1 # все что равно 0 становится -1 (так получим контрастное изображение)
    return x

# Из вектора обратно в изображение
def array2img(data, outFile = None, show = True):
    
    # инициализируем матрицу из нулей
    y = np.zeros(data.shape,dtype=np.uint8)
    # преобразуем данные из 1 и -1 в значения канала
    y[data==1] = 255
    y[data==-1] = 0
    print("Матрица в формате изображения — для наглядности изменений:\n", y)
    img = Image.fromarray(y,mode="L")
    if show == True:
        img.show()
    if outFile is not None:
        img.save(outFile)
    return img

def array2colorCode(data, outFile = None):
    
    # инициализируем матрицу из нулей
    y = np.zeros(data.shape,dtype=np.uint8)
    # преобразуем данные из 1 и -1 в значения канала
    y[data==1] = 255
    y[data==-1] = 0

    return y

# востанавливаем зашумленный образ
def update(w,y_vec,theta=0.5):
    global t
    t = False
    
    # Остановка когда зашумленный вектор после очередного обновления не будет равен предыдущему своему состоянию
    while t == False:
        
        y_vec1 = copy.copy(y_vec) # создаем копию зашумленного вектора до его изменения
        for s in range(1, len(y_vec)+1):
            m = len(y_vec)
            i = random.randint(0,m-1) # случайным выбирает значение в пределах размера восстанавливаемого образца
            u = np.dot(w[i][:],y_vec) - theta # перемножаем случайный i-й столбец с вектором образца

            if u >= 0: # фунция активации
                y_vec[i] = 1
            elif u < 0:
                y_vec[i] = -1
                
        t = (y_vec==y_vec1).all() # сравниваем измененный вектор с его предыдущим состоянием, если True выходим из цикла и начинаем три теста
        print(t)
        # print(array2colorCode(y_vec.reshape(oshape))) # показывает измененный вектор после каждого прохода по нему
        
    t = False
    for test in range(1, 4):
        for t in range(1, len(y_vec)+1):
            m = len(y_vec)
            i = random.randint(0,m-1) # случайным выбирает значение в пределах размера восстанавливаемого образца
            u = np.dot(w[i][:],y_vec) - theta # перемножаем случайный i-й столбец с вектором образца
        
            if u >= 0: # фунция активации
                y_vec[i] = 1
            elif u < 0:
                y_vec[i] = -1
                
            t = (y_vec==y_vec1).all()
        print("Тест №:", test, "\n", array2colorCode(y_vec.reshape(oshape)), y_vec.shape, t, "\n")
    
    if t == False:
        print("Обновляем еще раз")
        update(w, y_vec, theta=0.5)
    else:
        print("Вектор восстановленный и вектор исходный равны, ура!", "Это тому подверждение - здесь функция сравнивает эти векторы и отдает соотвественно:", (y_vec==y_vec1).all())
        y_vec = y_vec.reshape(oshape)
    
    return y_vec

## Обучение и восстановление

In [5]:
def hopfield(train_files, test_files,theta=0.5, invert=False, size=(100,100),threshold=60, current_path=None):
        
    # загрузка исходного образца
    # получение изображение и преобразование его в вектор
    print("Импорт изображения и создание матрицы весов...")

    # num_files - количество файлов
    num_files = 0
    for path in train_files:
        print(path)
        x = readImg2array(file=path,size=size,threshold=threshold,invert=invert)
        
        print("Оригинальная матрица преобразованная №", num_files+1, "\n", array2colorCode(x, x.shape), x.shape)
        x_vec = mat_to_vec(x)
        print("Длина получившегося вектора:", len(x_vec), "\n")
        if num_files == 0:
            w = create_W(x_vec)
            num_files = 1
        else:
            tmp_w = create_W(x_vec)
            w += tmp_w
            num_files +=1

    print("Матрица весов: \n", w, w.shape)
    
    
    
    # Импорт восстанавливаемых изображений
    counter = 0
    for path in test_files:
        
        y = readImg2array(file=path,size=size,threshold=threshold) # преобразуем изображение в матрицу чисел
        
#        y = np.array(pd.read_csv(path, dtype=np.float32, sep='\t', header = None))
        
        print("\n\nВосстанавливаемые  данные загружены.")
        print(path)
        print("\nМатрица №", counter+1)
        print("Зашумленная матрица\n", y)
        
        global oshape
        oshape = y.shape
        y_img = array2img(y)

        y_vec = mat_to_vec(y) # преобразуем матрицу чисел в один длинный вектор строка за строкой
        
        print("\nОбновление...")
        
        y_vec_after = update(w=w,y_vec=y_vec,theta=theta) # само восстановление зашумленного образца
        
        y_vec_after = y_vec_after.reshape(oshape) # из вектора обратно в матрицу

        
        if current_path is not None: # сохраняем восстановленное изображение в файл "after_*.jpg"
            outfile = current_path+"/after_"+str(counter+1)+".jpeg"
            array2img(y_vec_after,outFile=outfile)
        else:
            after_img = array2img(y_vec_after,outFile=None, show=False)
            after_img.show()
        counter +=1
        
        
# Исходные данные
current_path = os.getcwd()
train_paths = []
path = current_path+"/train_pics/"
for i in os.listdir(path):
    if re.match(r'[0-9a-zA-Z-]*.jp[e]*g',i):
        train_paths.append(path+i)

# Зашумленные
test_paths = []
path = current_path+"/test_pics/"
for i in os.listdir(path):
#    if re.match(r'[0-9a-zA-Z-_]*.txt',i):
    if re.match(r'[0-9a-zA-Z-_]*.jp[e]*g',i):
        test_paths.append(path+i)

## Для ввода в матричном виде

In [7]:
# Запуск

hopfield(train_files=train_paths,
           test_files=test_paths,
           theta=0.5,
           invert=False,
           size=(16,16),
           threshold=60,
           current_path = current_path)

Импорт изображения и создание матрицы весов...
/Users/germankucheravenko/Documents/python/Интеграция ИТ в ИС/Hopfield/Hopfield_net/train_pics/2.jpg
Оригинальная матрица преобразованная № 1 
 [[255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255   0   0   0   0   0   0   0   0   0   0   0 255 255]
 [255 255 255   0   0   0   0   0   0   0   0   0   0   0 255 255]
 [255 255 255   0   0 255 255 255 255 255 255 255   0   0 255 255]
 [255 255 255   0   0 255 255 255 255 255 255 255   0   0 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255   0   0 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255   0   0 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255   0   0 255 255]
 [255 255 255   0   0   0   0   0   0   0   0   0   0   0 255 255]
 [255 255 255   0   0   0   0   0   0   0   0   0   0   0 255 255]
 [255 255 255   0   0 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255   0   0 255 255 255 255 255 255 255 255 255 255 255]
 [255


Обновление...
False
False
False
False
True
Тест №: 1 
 [[255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255   0   0   0   0   0   0   0   0   0   0   0 255 255]
 [255 255 255   0   0   0   0   0   0   0   0   0   0   0 255 255]
 [255 255 255   0 255 255 255 255 255 255 255 255   0   0 255 255]
 [255 255 255   0 255 255 255 255 255 255 255 255   0   0 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255   0   0 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255   0   0 255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255   0   0 255 255]
 [255 255 255   0 255 255 255   0   0 255 255 255   0   0 255 255]
 [255 255 255   0 255 255 255   0   0 255 255 255   0   0 255 255]
 [255 255 255   0 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255   0   0 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255   0   0 255 255 255 255 255 255 255 255 255 255 255]
 [255 255 255   0   0   0   0   0   0   0   0   0   0   0 255 255]
 [255 